[View in Colaboratory](https://colab.research.google.com/github/rdenadai/TxtP-Study-Notebooks/blob/master/notebooks/text_classification_example.ipynb)

## Análise e Validação de Textos em Português


### Referências:

 - [NLTK](http://www.nltk.org/howto/portuguese_en.html)
 - [spaCy](https://spacy.io/usage/spacy-101)
 - [Utilizando processamento de linguagem natural para criar um sumarização automática de textos](https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e)

### Instalação

In [1]:
!pip install -U spacy
!python -m spacy download en
!python -m spacy download pt

Requirement already up-to-date: spacy in /usr/local/lib/python2.7/dist-packages (2.0.12)
    100% |████████████████████████████████| 37.4MB 1.1MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python2.7/dist-packages/en_core_web_sm -->
    /usr/local/lib/python2.7/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 38.7MB 72.1MB/s 
  Running setup.py install for pt-core-news-sm ... - \ done

    Linking successful
    /usr/local/lib/python2.7/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python2.7/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')



### Imports

In [3]:
import nltk
import spacy

nlp = spacy.load('pt')

nltk.download('averaged_perceptron_tagger')
nltk.download('floresta')
nltk.download('mac_morpho')
nltk.download('machado')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Package floresta is already up-to-date!
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### Usage

In [5]:
doc = nlp(u"Gostaria de saber mais informações sobre a Amazon.")
print([(w.text, w.pos_) for w in doc])

[(u'Gostaria', u'VERB'), (u'de', u'ADP'), (u'saber', u'VERB'), (u'mais', u'DET'), (u'informa\xe7\xf5es', u'NOUN'), (u'sobre', u'ADP'), (u'a', u'DET'), (u'Amazon', u'PROPN'), (u'.', u'PUNCT')]
